In [ ]:
import tensorflow as tf
from dh_segment.inference.loader import LoadedModel
from glob import glob
import numpy as np
from tqdm import tqdm
import os
import cv2
from typing import List
from imageio import imsave

In [ ]:
from dh_segment.loader import LoadedModel
from process import prediction_fn

In [ ]:
model_dirs_list = ['../../dataset/model2/export/1562877546/']
input_dir = '../../dataset/cb55/val/images/'
output_dir_name = 'out_predictions'

## Prediction

In [ ]:
for model_dir in model_dirs_list:
    output_dir = '{}'.format(os.path.sep).join(model_dir.split(os.path.sep)[:-3] + [output_dir_name])
    os.makedirs(output_dir, exist_ok=True)
    prediction_fn(model_dir, input_dir, output_dir)

## Evaluation

In [ ]:
from .evaluation import eval_fn

In [ ]:
pred_dir_list = [os.path.abspath(os.path.join(md, '..', '..', output_dir_name)) 
                 for md in model_dirs_list]
gt_dir = 'diva_test_set/pixel-level-gt/'
DIVA_JAR = 'DIVA_Layout_Analysis_Evaluator/out/artifacts/LayoutAnalysisEvaluator.jar'

In [ ]:
list_mIUs = list()
for i, pred_dir in enumerate(pred_dir_list):
    output_txt_filename = './{i}_diva_results.txt'
    mean_mIU = eval_fn(pred_dir, gt_dir, output_txt_filename, diva_jar=DIVA_JAR)
    list_mIUs.append(mean_mIU)

In [ ]:
print('MIOU : {:.03} +- {:.03} ([{:.03}, {:.03}])'.format(np.mean(list_mIUs), 
                                                    np.std(list_mIUs), 
                                                    np.min(list_mIUs), 
                                                    np.max(list_mIUs)))